## 1. Setup

Verify environment and import required libraries.

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Check Python version
print(f"Python version: {sys.version}")
assert sys.version_info >= (3, 10), "Python 3.10+ required"

# Import core libraries
import numpy as np
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, Video
import json
from datetime import datetime

print(f"\nPyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"OpenCV version: {cv2.__version__}")

In [ ]:
# Check CUDA availability
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Print memory stats
    print(f"\nGPU Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"GPU Memory cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
else:
    print("\n⚠️  WARNING: No GPU detected. Heavy inference steps will be skipped.")
    print("   For full functionality, run on a machine with NVIDIA GPU + CUDA.")

## 2. Configuration

Set all parameters in one place.

In [ ]:
# ============================================
# CONFIGURATION CELL - Edit parameters here
# ============================================

# Case/experiment name
CASE_NAME = "dog_test_01"

# Paths
PROJECT_ROOT = Path("../").resolve()
INPUT_DIR = PROJECT_ROOT / "data" / "raw" / CASE_NAME
OUTPUT_DIR = PROJECT_ROOT / "outputs" / CASE_NAME

# Image processing
MAX_IMAGES = 50  # Maximum images to process
IMAGE_SIZE = 512  # Target image size for processing
MIN_IMAGE_SIZE = 256  # Minimum acceptable image size

# Quality thresholds
BLUR_THRESHOLD = 100.0  # Laplacian variance threshold (lower = more blurry)
PERCEPTUAL_HASH_THRESHOLD = 5  # Hamming distance for duplicate detection

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Model configuration
HUNYUAN_MODEL_ID = "tencent/Hunyuan3D-2"
ENABLE_TEXTURE = True  # Set to False to skip texture generation (saves VRAM)

# Output settings
TURNTABLE_FRAMES = 36  # Number of frames for turntable video
TURNTABLE_FPS = 12  # Frames per second for turntable video

# Display configuration
print(f"Configuration:")
print(f"  Case name: {CASE_NAME}")
print(f"  Input dir: {INPUT_DIR}")
print(f"  Output dir: {OUTPUT_DIR}")
print(f"  Device: {DEVICE}")
print(f"  Image size: {IMAGE_SIZE}px")
print(f"  Texture generation: {'Enabled' if ENABLE_TEXTURE else 'Disabled'}")

In [ ]:
# Create output directories
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "images").mkdir(exist_ok=True)
(OUTPUT_DIR / "meshes").mkdir(exist_ok=True)
(OUTPUT_DIR / "renders").mkdir(exist_ok=True)

print(f"✓ Output directories created at: {OUTPUT_DIR}")

## 3. Load Images

Load all images from the input directory.

In [ ]:
# Check if input directory exists
if not INPUT_DIR.exists():
    print(f"⚠️  Input directory does not exist: {INPUT_DIR}")
    print(f"\nPlease create the directory and add images:")
    print(f"  mkdir -p {INPUT_DIR}")
    print(f"  # Then copy your .jpg/.png images to {INPUT_DIR}")
    raise FileNotFoundError(f"Input directory not found: {INPUT_DIR}")

# Find all image files
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
image_files = []

for ext in image_extensions:
    image_files.extend(list(INPUT_DIR.glob(f"*{ext}")))

image_files = sorted(image_files)[:MAX_IMAGES]

print(f"Found {len(image_files)} images in {INPUT_DIR}")

if len(image_files) == 0:
    print(f"\n⚠️  No images found in {INPUT_DIR}")
    print(f"   Supported formats: {', '.join(image_extensions)}")
    raise ValueError("No images found")

# Display first few filenames
print(f"\nFirst images:")
for i, img_path in enumerate(image_files[:5]):
    print(f"  {i+1}. {img_path.name}")
    
if len(image_files) > 5:
    print(f"  ... and {len(image_files) - 5} more")

In [ ]:
# Load images and get basic stats
images = []
image_stats = []

print("Loading images...")
for img_path in image_files:
    try:
        img = Image.open(img_path).convert('RGB')
        img_array = np.array(img)
        
        images.append(img)
        image_stats.append({
            'path': img_path,
            'name': img_path.name,
            'width': img.width,
            'height': img.height,
            'mean_brightness': img_array.mean(),
            'size_bytes': img_path.stat().st_size
        })
    except Exception as e:
        print(f"  ⚠️  Failed to load {img_path.name}: {e}")

print(f"✓ Loaded {len(images)} images successfully")

# Display resolution statistics
widths = [s['width'] for s in image_stats]
heights = [s['height'] for s in image_stats]

print(f"\nResolution statistics:")
print(f"  Width:  {np.min(widths)} - {np.max(widths)} px (mean: {np.mean(widths):.0f})")
print(f"  Height: {np.min(heights)} - {np.max(heights)} px (mean: {np.mean(heights):.0f})")

In [ ]:
# Visualize sample images
n_display = min(9, len(images))
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

for i in range(n_display):
    axes[i].imshow(images[i])
    axes[i].set_title(f"{image_stats[i]['name'][:20]}\n{image_stats[i]['width']}x{image_stats[i]['height']}px")
    axes[i].axis('off')

for i in range(n_display, 9):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "images" / "input_grid.png", dpi=100, bbox_inches='tight')
plt.show()

print(f"✓ Saved input grid to: {OUTPUT_DIR / 'images' / 'input_grid.png'}")

## 4. Preprocess & Quality Filtering

Run quality checks on images:
- Size filtering (remove too-small images)
- Blur detection (Laplacian variance)
- Duplicate detection (perceptual hashing)

In [ ]:
# Helper function: compute blur score
def compute_blur_score(image):
    """Compute Laplacian variance as blur metric. Higher = sharper."""
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var

# Helper function: compute perceptual hash
def compute_phash(image, hash_size=8):
    """Compute perceptual hash for duplicate detection."""
    img_gray = image.convert('L').resize((hash_size, hash_size), Image.LANCZOS)
    pixels = np.array(img_gray).flatten()
    avg = pixels.mean()
    diff = pixels > avg
    return diff

# Helper function: hamming distance between hashes
def hamming_distance(hash1, hash2):
    """Compute hamming distance between two binary arrays."""
    return np.sum(hash1 != hash2)

print("Computing quality metrics...")

In [ ]:
# Add quality metrics to image stats
for i, (img, stats) in enumerate(zip(images, image_stats)):
    stats['blur_score'] = compute_blur_score(img)
    stats['phash'] = compute_phash(img)
    stats['quality_score'] = stats['blur_score']  # Can combine multiple factors

print(f"✓ Computed quality metrics for {len(image_stats)} images")

In [ ]:
# Filter by size
size_filtered = []
for stats in image_stats:
    min_dim = min(stats['width'], stats['height'])
    if min_dim >= MIN_IMAGE_SIZE:
        size_filtered.append(stats)
    else:
        print(f"  Filtered (too small): {stats['name']} ({stats['width']}x{stats['height']})")

print(f"\n✓ Size filter: {len(size_filtered)}/{len(image_stats)} images passed (min {MIN_IMAGE_SIZE}px)")

In [ ]:
# Filter by blur
blur_filtered = []
for stats in size_filtered:
    if stats['blur_score'] >= BLUR_THRESHOLD:
        blur_filtered.append(stats)
    else:
        print(f"  Filtered (blurry): {stats['name']} (score: {stats['blur_score']:.1f})")

print(f"\n✓ Blur filter: {len(blur_filtered)}/{len(size_filtered)} images passed (threshold: {BLUR_THRESHOLD})")

In [ ]:
# Filter duplicates
unique_images = []
seen_hashes = []

for stats in blur_filtered:
    is_duplicate = False
    for seen_hash in seen_hashes:
        if hamming_distance(stats['phash'], seen_hash) < PERCEPTUAL_HASH_THRESHOLD:
            is_duplicate = True
            print(f"  Filtered (duplicate): {stats['name']}")
            break
    
    if not is_duplicate:
        unique_images.append(stats)
        seen_hashes.append(stats['phash'])

print(f"\n✓ Duplicate filter: {len(unique_images)}/{len(blur_filtered)} unique images")
print(f"\n📊 Final dataset: {len(unique_images)} images")

In [ ]:
# Select best image (hero image) for 3D reconstruction
if len(unique_images) == 0:
    raise ValueError("No images passed quality filters!")

# Sort by quality score (blur score in this case)
unique_images_sorted = sorted(unique_images, key=lambda x: x['quality_score'], reverse=True)
hero_image_stats = unique_images_sorted[0]
hero_image_path = hero_image_stats['path']
hero_image = Image.open(hero_image_path).convert('RGB')

print(f"\n🌟 Best image selected: {hero_image_stats['name']}")
print(f"   Resolution: {hero_image_stats['width']}x{hero_image_stats['height']}")
print(f"   Quality score: {hero_image_stats['quality_score']:.1f}")
print(f"   Brightness: {hero_image_stats['mean_brightness']:.1f}")

# Display hero image
plt.figure(figsize=(8, 8))
plt.imshow(hero_image)
plt.title(f"Hero Image: {hero_image_stats['name']}")
plt.axis('off')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "images" / "hero_image.png", dpi=150, bbox_inches='tight')
plt.show()

## 5. Hunyuan3D-2 Inference

Run image-to-3D generation using Hunyuan3D-2 model.

In [ ]:
if DEVICE == "cpu":
    print("⚠️  WARNING: Running on CPU. Skipping Hunyuan3D-2 inference.")
    print("   This step requires a CUDA-enabled GPU.")
    print("   Set DEVICE='cuda' and ensure PyTorch with CUDA is installed.")
    SKIP_INFERENCE = True
else:
    SKIP_INFERENCE = False
    print(f"✓ GPU detected. Proceeding with inference on {DEVICE}")

In [ ]:
if not SKIP_INFERENCE:
    print("Installing/importing Hunyuan3D-2 dependencies...")
    print("Note: This may take a few minutes on first run.\n")
    
    try:
        # Import Hugging Face libraries
        from transformers import AutoModel, AutoProcessor
        from huggingface_hub import hf_hub_download
        import trimesh
        
        print("✓ Dependencies imported successfully")
        
    except ImportError as e:
        print(f"❌ Missing dependencies: {e}")
        print("\nPlease install required packages:")
        print("  pip install transformers huggingface_hub trimesh")
        SKIP_INFERENCE = True

In [ ]:
if not SKIP_INFERENCE:
    print(f"Loading Hunyuan3D-2 model: {HUNYUAN_MODEL_ID}")
    print("This may download several GB on first run...\n")
    
    try:
        # Note: The actual API may differ. This is a template.
        # Check official Hunyuan3D-2 documentation for exact usage.
        
        # Placeholder for model loading
        print("⚠️  Model loading code is a template.")
        print("   Please refer to Hunyuan3D-2 documentation for exact API.")
        print("   Expected: AutoModel.from_pretrained() or similar\n")
        
        # Example structure (may need adjustment):
        # model = AutoModel.from_pretrained(HUNYUAN_MODEL_ID)
        # processor = AutoProcessor.from_pretrained(HUNYUAN_MODEL_ID)
        # model.to(DEVICE)
        # model.eval()
        
        MODEL_LOADED = False  # Set to True when model loads successfully
        
        print("✓ Model loading structure ready (needs implementation)")
        
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        SKIP_INFERENCE = True
        MODEL_LOADED = False

In [ ]:
if not SKIP_INFERENCE and MODEL_LOADED:
    print("Running shape generation inference...\n")
    
    try:
        # Preprocess hero image
        # Resize to target size while maintaining aspect ratio
        hero_image_resized = hero_image.copy()
        hero_image_resized.thumbnail((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
        
        print(f"Processing image: {hero_image_resized.size}")
        
        # Run inference (API template - adjust based on actual Hunyuan3D-2 API)
        print("⚠️  Inference code is a template. Adjust for actual API.\n")
        
        # Example structure:
        # inputs = processor(images=hero_image_resized, return_tensors="pt").to(DEVICE)
        # with torch.no_grad():
        #     outputs = model.generate(**inputs, num_inference_steps=50)
        # mesh = outputs.mesh  # or similar
        
        print("Shape generation: [TEMPLATE - needs implementation]")
        
        SHAPE_GENERATED = False  # Set to True when shape is generated
        
    except Exception as e:
        print(f"❌ Inference failed: {e}")
        SHAPE_GENERATED = False
else:
    print("Skipping inference (no GPU or model not loaded)")
    SHAPE_GENERATED = False

In [ ]:
if not SKIP_INFERENCE and MODEL_LOADED and SHAPE_GENERATED and ENABLE_TEXTURE:
    print("Running texture generation (optional)...\n")
    
    # Check if enough VRAM available
    if torch.cuda.is_available():
        free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)
        free_gb = free_memory / 1e9
        
        print(f"Available GPU memory: {free_gb:.2f} GB")
        
        if free_gb < 4.0:
            print("⚠️  Low VRAM. Skipping texture generation.")
            TEXTURE_GENERATED = False
        else:
            try:
                # Run texture generation (API template)
                print("Texture generation: [TEMPLATE - needs implementation]\n")
                
                # Example:
                # textured_mesh = model.add_texture(mesh, hero_image_resized)
                
                TEXTURE_GENERATED = False  # Set to True when texture is added
                
            except Exception as e:
                print(f"❌ Texture generation failed: {e}")
                TEXTURE_GENERATED = False
    else:
        TEXTURE_GENERATED = False
else:
    print("Skipping texture generation")
    TEXTURE_GENERATED = False

## 6. Export & Visualization

Save generated 3D mesh to standard formats.

In [ ]:
if SHAPE_GENERATED:
    print("Exporting mesh...\n")
    
    # Export to OBJ format
    obj_path = OUTPUT_DIR / "meshes" / f"{CASE_NAME}_mesh.obj"
    
    try:
        # Export mesh (API template)
        print(f"⚠️  Export code is a template.\n")
        
        # Example:
        # mesh.export(obj_path)
        # or
        # trimesh_obj = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
        # trimesh_obj.export(obj_path)
        
        print(f"Would save mesh to: {obj_path}")
        
        # Try GLB/GLTF export if supported
        glb_path = OUTPUT_DIR / "meshes" / f"{CASE_NAME}_mesh.glb"
        print(f"Would save GLB to: {glb_path}")
        
        MESH_EXPORTED = False  # Set to True when export succeeds
        
    except Exception as e:
        print(f"❌ Export failed: {e}")
        MESH_EXPORTED = False
else:
    print("No mesh to export (inference was skipped)")
    MESH_EXPORTED = False

In [ ]:
# Create a simple metadata file
metadata = {
    'case_name': CASE_NAME,
    'timestamp': datetime.now().isoformat(),
    'input_images': len(image_files),
    'filtered_images': len(unique_images),
    'hero_image': hero_image_stats['name'],
    'device': DEVICE,
    'model': HUNYUAN_MODEL_ID,
    'image_size': IMAGE_SIZE,
    'shape_generated': SHAPE_GENERATED,
    'texture_generated': TEXTURE_GENERATED,
    'mesh_exported': MESH_EXPORTED
}

metadata_path = OUTPUT_DIR / "metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Saved metadata to: {metadata_path}")

## 7. Turntable Render

Generate orbit frames and create turntable video.

In [ ]:
if MESH_EXPORTED:
    print("Generating turntable renders...\n")
    
    try:
        import pyrender
        from pyrender import RenderFlags
        
        print("✓ PyRender available")
        PYRENDER_AVAILABLE = True
        
    except ImportError:
        print("⚠️  PyRender not available. Install with: pip install pyrender")
        print("   Falling back to basic rendering...\n")
        PYRENDER_AVAILABLE = False
else:
    print("No mesh available for turntable rendering")
    PYRENDER_AVAILABLE = False

In [ ]:
if MESH_EXPORTED and PYRENDER_AVAILABLE:
    print(f"Rendering {TURNTABLE_FRAMES} orbit frames...\n")
    
    try:
        # Load mesh
        # mesh_obj = trimesh.load(obj_path)
        
        # Setup PyRender scene (template)
        print("⚠️  Rendering code is a template.\n")
        
        # Example structure:
        # scene = pyrender.Scene()
        # mesh_node = pyrender.Mesh.from_trimesh(mesh_obj)
        # scene.add(mesh_node)
        
        # camera_pose = np.eye(4)
        # camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        # scene.add(camera, pose=camera_pose)
        
        # light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
        # scene.add(light, pose=camera_pose)
        
        # renderer = pyrender.OffscreenRenderer(512, 512)
        
        render_frames = []
        
        for i in range(TURNTABLE_FRAMES):
            angle = 2 * np.pi * i / TURNTABLE_FRAMES
            
            # Rotate camera around object
            # Update camera pose based on angle
            # color, depth = renderer.render(scene)
            # render_frames.append(color)
            
            if i % 10 == 0:
                print(f"  Rendered frame {i+1}/{TURNTABLE_FRAMES}")
        
        print(f"\n✓ Rendered {TURNTABLE_FRAMES} frames")
        FRAMES_RENDERED = False  # Set to True when rendering succeeds
        
    except Exception as e:
        print(f"❌ Rendering failed: {e}")
        FRAMES_RENDERED = False
else:
    print("Skipping turntable rendering")
    FRAMES_RENDERED = False

In [ ]:
if FRAMES_RENDERED:
    print("Creating turntable video...\n")
    
    try:
        import imageio
        
        video_path = OUTPUT_DIR / "turntable.mp4"
        
        # Write video
        # writer = imageio.get_writer(video_path, fps=TURNTABLE_FPS)
        # for frame in render_frames:
        #     writer.append_data(frame)
        # writer.close()
        
        print(f"✓ Turntable video would be saved to: {video_path}")
        print(f"   {TURNTABLE_FRAMES} frames @ {TURNTABLE_FPS} FPS")
        
        # Display video in notebook
        # display(Video(video_path, embed=True, width=600))
        
    except Exception as e:
        print(f"❌ Video creation failed: {e}")
        print("   Install imageio: pip install imageio[ffmpeg]")

## 8. Nerfstudio Notes (Optional)

For NeRF-based reconstruction, you can use **Nerfstudio** with your filtered image set.

### Installation

```bash
pip install nerfstudio
```

### Basic workflow

1. **Process images with COLMAP** (structure-from-motion):
   ```bash
   ns-process-data images --data data/raw/dog_test_01 --output-dir data/processed/dog_test_01
   ```

2. **Train NeRF model** (e.g., nerfacto):
   ```bash
   ns-train nerfacto --data data/processed/dog_test_01
   ```

3. **Export mesh**:
   ```bash
   ns-export poisson --load-config outputs/dog_test_01/nerfacto/config.yml --output-dir outputs/dog_test_01/mesh
   ```

4. **View in browser**:
   - Training automatically launches viewer at `http://localhost:7007`

### Requirements for NeRF

- Multiple views (10+ images) from different angles
- Good camera coverage around the object
- Consistent lighting
- Static scene (no moving objects)

### Comparison: Single-image 3D vs. NeRF

| Method | Images | Quality | Speed | Use Case |
|--------|--------|---------|-------|----------|
| Hunyuan3D-2 | 1 | Good | Fast | Quick previews, single-view data |
| NeRF | 10-100+ | Excellent | Slow | High-quality, multi-view data |

For this smoketest, we focus on **Hunyuan3D-2** for single-image reconstruction.

## 9. Summary

Generate final report.

In [ ]:
print("="*60)
print(" SMOKETEST SUMMARY")
print("="*60)
print(f"\nCase: {CASE_NAME}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nEnvironment:")
print(f"  Device: {DEVICE}")
print(f"  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
print(f"\nData:")
print(f"  Input images: {len(image_files)}")
print(f"  After filtering: {len(unique_images)}")
print(f"  Hero image: {hero_image_stats['name']}")
print(f"\nPipeline Results:")
print(f"  ✓ Quality checks passed")
print(f"  {'✓' if SHAPE_GENERATED else '✗'} Shape generation")
print(f"  {'✓' if TEXTURE_GENERATED else '✗'} Texture generation")
print(f"  {'✓' if MESH_EXPORTED else '✗'} Mesh export")
print(f"  {'✓' if FRAMES_RENDERED else '✗'} Turntable rendering")
print(f"\nOutputs saved to: {OUTPUT_DIR}")
print(f"  - metadata.json")
print(f"  - images/")
print(f"  - meshes/")
print(f"  - renders/")
if FRAMES_RENDERED:
    print(f"  - turntable.mp4")
print("\n" + "="*60)

if not SHAPE_GENERATED:
    print("\n⚠️  NOTE: Full pipeline requires GPU with CUDA")
    print("   This run completed quality checks only.")
    print("   Re-run on GPU-enabled machine for 3D generation.")